In [1]:
'''
What’s the percentage of users, who reviewed ​“Mon Ami Gabi”,
and also reviewed at least 10 other restaurants located in Ontario?

users who reviewed MAG and review 10+ Ontario restaurants
 /
 all users

'''

from pymongo import MongoClient
import pandas as pd
import pprint


client = MongoClient()
dblist = client.list_database_names()
db = client.yelp
pp = pprint.PrettyPrinter(indent=4)




In [2]:
cursor = db.review.find({})

In [3]:
all_reviews = list(cursor)

# Find Users who have reviewed MAG

## Find Reviews for MAG

In [4]:
df_reviews = pd.DataFrame(all_reviews)

In [5]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6685900 entries, 0 to 6685899
Data columns (total 10 columns):
_id            object
review_id      object
user_id        object
business_id    object
stars          float64
useful         int64
funny          int64
cool           int64
text           object
date           object
dtypes: float64(1), int64(3), object(6)
memory usage: 510.1+ MB


In [6]:
df_reviews = df_reviews[['review_id', 'user_id', 'business_id']]

In [8]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6685900 entries, 0 to 6685899
Data columns (total 3 columns):
review_id      object
user_id        object
business_id    object
dtypes: object(3)
memory usage: 153.0+ MB


In [10]:
# FIND MAG BUSINESS ID
mon_ami_gabi = db.business.find_one({"name": "Mon Ami Gabi"}, {"name": 1, "business_id": 1, "_id": 1, "city": 1})
mag_business_id = mon_ami_gabi["business_id"]
mag_business_id

'4JNXUYY8wbaaDmk3BPzlWw'

In [11]:
df_mag_reviews = df_reviews[df_reviews['business_id'] == mag_business_id]

In [12]:
df_mag_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8570 entries, 2661165 to 3331303
Data columns (total 3 columns):
review_id      8570 non-null object
user_id        8570 non-null object
business_id    8570 non-null object
dtypes: object(3)
memory usage: 267.8+ KB


In [16]:
df_mag_reviewer_user_id = pd.DataFrame(df_mag_reviews['user_id'].unique())

In [17]:
df_mag_reviewer_user_id.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8349 entries, 0 to 8348
Data columns (total 1 columns):
0    8349 non-null object
dtypes: object(1)
memory usage: 65.4+ KB


In [18]:
df_mag_reviewer_user_id.head()

,0
0,x5NpvWNS1cF_4bINIBPGzQ
1,m5cBT7OYfn0HM0d-RCsdZQ
2,ANsHMXjqFUTvZs0KQ7RB5Q
3,2L4W7nak0F6AxFAeBhFI1w
4,Oj_66IjmwmVdZTIz5xZXWg


In [150]:
set_mag_reviewers = set(df_mag_reviewer_user_id[0].to_list())

# Find users with more than 10 reviews for restaurants in Ontario

In [35]:
## Find Ontario businesses

In [38]:
cursor_on_business = db.business.find({"state": "ON"}, {"business_id": 1, "_id":0})
df_on_business = pd.DataFrame(list(cursor_on_business))
df_on_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33412 entries, 0 to 33411
Data columns (total 1 columns):
business_id    33412 non-null object
dtypes: object(1)
memory usage: 261.2+ KB


In [40]:
set_on_business = set(df_on_business['business_id'])

In [41]:
df_reviews['is_on_review'] = df_reviews['business_id'].apply(lambda id: id in set_on_business)

/Users/leeseunghoon/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [94]:
df_on_reviews = df_reviews[df_reviews['is_on_review'] == True]
# df_on_reviews = df_on_reviews['user_id']

In [131]:
df_on_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 784461 entries, 8 to 6685880
Data columns (total 4 columns):
review_id       784461 non-null object
user_id         784461 non-null object
business_id     784461 non-null object
is_on_review    784461 non-null bool
dtypes: bool(1), object(3)
memory usage: 24.7+ MB


In [132]:
df_on_reviews.head()

,review_id,user_id,business_id,is_on_review
8,G7XHMxG0bx9oBJNECG4IFg,jlu4CztcSxrKx56ba1a5AQ,3fw2X5bZYeW9xCz_zGhOHg,True
11,JVcjMhlavKKn3UIt9p9OXA,TpyOT5E16YASd7EWjLQlrw,AakkkTuGZA2KBodKi2_u8A,True
14,svK3nBU7Rk8VfGorlrN52A,NJlxGtouq06hhC7sS2ECYw,YvrylyuWgbP90RgMqZQVnQ,True
20,4bUyL7lzoWzDZaJETAKREg,_N7Ndn29bpll_961oPeEfw,y-Iw6dZflNix4BdwIyTNGA,True
32,E6B-2U2sGG3xgmnNWZAEew,DbccYu3OppWKl21OanZnTg,YSUcHqlKMPHHJ_cTrqtNrA,True


In [133]:
df_on_reviews_by_user = df_on_reviews.groupby("user_id").count()

In [134]:
df_on_reviews_by_user.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148570 entries, --56y1InAvNoQOD6YYrhVQ to zzyMMeUZzKAy7KQhM7lU2w
Data columns (total 3 columns):
review_id       148570 non-null int64
business_id     148570 non-null int64
is_on_review    148570 non-null int64
dtypes: int64(3)
memory usage: 4.5+ MB


In [135]:
df_on_reviews_by_user.head()

,review_id,business_id,is_on_review
user_id,,,
--56y1InAvNoQOD6YYrhVQ,1,1,1
--6_wnx2sD1rqOQAQH96wg,1,1,1
--7gjElmOrthETJ8XqzMBw,6,6,6
--Br-QsbO9ad5GbZxVGxaw,8,8,8
--BumyUHiO_7YsHurb9Hkw,55,55,55


In [137]:
df_on_reviews_by_user = df_on_reviews_by_user[df_on_reviews_by_user['review_id'] >= 10]

In [138]:
df_on_reviews_by_user.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13937 entries, --BumyUHiO_7YsHurb9Hkw to zzo--VpSQh8PpsGVeMC1dQ
Data columns (total 3 columns):
review_id       13937 non-null int64
business_id     13937 non-null int64
is_on_review    13937 non-null int64
dtypes: int64(3)
memory usage: 435.5+ KB


In [139]:
df_on_reviews_by_user.head()

,review_id,business_id,is_on_review
user_id,,,
--BumyUHiO_7YsHurb9Hkw,55,55,55
--Qh8yKWAvIP4V4K8ZPfHA,411,411,411
--UOvCH5qEgdNQ8lzR8QYQ,12,12,12
--YhjyV-ce1nFLYxP49C5A,87,87,87
-0AyZxS5C--WySnbW_Q8yQ,20,20,20


In [146]:
set_users_with_10_on_reviews = set(df_on_reviews_by_user.index.values.tolist())

In [153]:
answer = set_mag_reviewers.intersection(set_users_with_10_on_reviews)

In [154]:
answer

{'-hUgrj7Lzir3yLUYrMYQ4g',
 '0HxInQ94hVHVlO1FGPWctA',
 '1O638BDK_fWuxgTVJwff-A',
 '2HiNttKgEkrwJQUTrL3Hxw',
 '41kMyI1BfOTyIIAenUSaTQ',
 '4T8X9Nss9bV6I_AdHNmTYQ',
 '4ayVX0YKY3GSP2IICpLJXQ',
 '4w-xY0Yc-jOvoCT4vJucUQ',
 '56_DKQpK4tT0rWLt4-DZOQ',
 '7JAdkaWbXuXA4YULqxKaMg',
 '7cmEMOLyD4GS1XbjctJpoQ',
 '8RcEwGrFIgkt9WQ35E6SnQ',
 'AKHX3VbnKrdE_nIOCd5ptA',
 'B856VSGCvHAB-mil7JVprA',
 'C6J6aogoZ5bXWOZ76Mg36A',
 'DBHCFW3mSmmOEpONHVu1rQ',
 'ENgL9AxV7uu-v5EUCpF69Q',
 'EiT_pSuIb08oRn4CfwfMJg',
 'F2cMAsmFfxT1iy92fVCmFw',
 'GNZE2VP4I2OsAu2D2PKhqg',
 'GlxJs5r01_yqIgb4CYtiog',
 'H7pj7sbXY3N-WSEwa-JfpA',
 'HFECrzYDpgbS5EmTBtj2zQ',
 'HkJ1p-QlRx6FJztgjr4Vnw',
 'HkKyQY5bh57z7vhmFAZj5w',
 'ITYKS6alUPmi8ubbNn7KRA',
 'Jf3EkyDJzpHG5d37iacdpw',
 'Lb37hatAjI3fgHEh2lMNEw',
 'ME52kxq4f7LfxUHg86Cjew',
 'MpvqV7lQcl15rflTBEUhXA',
 'NBNIUJj60tVBKvWHhy3egg',
 'O3eLhBJtr-L1ZCTnxjeIlg',
 'P4Td3m5oMGZGTfDVZzgztg',
 'PB3OEIzQGx_p6VdpzX3Feg',
 'Q1ZbMYWMdXlWPHpg24f9pw',
 'Qb8sDTzr54wRtT9hm1ZBCg',
 'Rvn-Ymc8_9z4vbcm2ZwenQ',
 

In [156]:
len(answer) / 1637138 * 100

0.005069823069283102